# Overview
Use pre-trained neural network to predict the existence and use it as the backend of the U-net model to do segmentation.
### Models used
We start from navie CNN, to models in keras.applications lib. We have tried navie CNN, VGG16, InceptionV3, Inception ResNet V2, Xception, DenseNet169, DenseNet121, ResNet50.

The models were trained on 15000 (80% train, 20% validation) images for 30 epoches. The best result is with ResNet50, with batch size 64, learning rate 1e-4, dropout 0.2. The binary accuray is about 92% for both training set and validation set.


### Model Parameters
Tried several parameters, the selected one are the best we know.

### Import library

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.util.montage import montage2d as montage
from sklearn.model_selection import train_test_split
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
ship_dir = '../input'
train_image_dir = os.path.join(ship_dir, 'train_v2')
test_image_dir = os.path.join(ship_dir, 'test_v2')
import gc; gc.enable() # memory is tight


### parameters setting

In [ ]:
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
VALID_IMG_COUNT = 1000
MAX_TRAIN_IMAGES = 15000 
IMG_SIZE = (299, 299) 
BATCH_SIZE = 64 
DROPOUT = 0.2
DENSE_COUNT = 128
LEARN_RATE = 1e-4
RGB_FLIP = 1 # should rgb be flipped when rendering images

In [ ]:
masks = pd.read_csv(os.path.join('../input/',
                                 'train_ship_segmentations_v2.csv'))
print(masks.shape[0], 'masks found')
print(masks['ImageId'].value_counts().shape[0])
masks['path'] = masks['ImageId'].map(lambda x: os.path.join(train_image_dir, x))
masks.head()

### Split into training and validation groups

In [ ]:
from sklearn.model_selection import train_test_split
masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()
unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x>0 else 0.0)
unique_img_ids['has_ship_vec'] = unique_img_ids['has_ship'].map(lambda x: [x])
masks.drop(['ships'], axis=1, inplace=True)
train_ids, valid_ids = train_test_split(unique_img_ids, 
                 test_size = 0.2, 
                 stratify = unique_img_ids['ships'])
train_df = pd.merge(masks, train_ids)
valid_df = pd.merge(masks, valid_ids)
print(train_df.shape[0], 'training masks')
print(valid_df.shape[0], 'validation masks')

### Examine Number of Ship Images
Here we examine how often ships appear and replace the ones without any ships with 0

In [ ]:
train_df = train_df.sample(min(MAX_TRAIN_IMAGES, train_df.shape[0])) # limit size of training set (otherwise it takes too long)

In [ ]:
train_df[['ships', 'has_ship']].hist()

## Augment Data
Since the number of images with ships and without ships are not even, we need to generate more data using ImageDataGenerator.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
dg_args = dict(featurewise_center = False, 
                  samplewise_center = False,
                  rotation_range = 45, 
                  width_shift_range = 0.1, 
                  height_shift_range = 0.1, 
                  shear_range = 0.01,
                  zoom_range = [0.9, 1.25],  
                  brightness_range = [0.5, 1.5],
                  horizontal_flip = True, 
                  vertical_flip = True,
                  fill_mode = 'reflect',
                   data_format = 'channels_last',
              preprocessing_function = preprocess_input)
valid_args = dict(featurewise_center = False,
              samplewise_center = False,
              rotation_range = 45, 
              width_shift_range = 0.1, 
              height_shift_range = 0.1, 
              shear_range = 0.01,
              zoom_range = [0.9, 1.25],  
              brightness_range = [0.5, 1.5],
              horizontal_flip = True, 
              vertical_flip = True,
              fill_mode = 'reflect',
               data_format = 'channels_last',
              preprocessing_function = preprocess_input)

core_idg = ImageDataGenerator(**dg_args)
valid_idg = ImageDataGenerator(**valid_args)

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [ ]:
train_gen = flow_from_dataframe(core_idg, train_df, 
                             path_col = 'path',
                            y_col = 'has_ship_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE)

# used a fixed dataset for evaluating the algorithm
valid_x, valid_y = next(flow_from_dataframe(valid_idg, 
                               valid_df, 
                             path_col = 'path',
                            y_col = 'has_ship_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = VALID_IMG_COUNT)) # one big batch
print(valid_x.shape, valid_y.shape)

In [ ]:
t_x, t_y = next(train_gen)
print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())

## Build a Model
We build the pre-trained top model and then use a global-max-pooling.

In [ ]:
from keras import models, layers
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D,Dropout,MaxPooling2D,Dense,Flatten
from keras.layers.normalization import BatchNormalization
tmodel_base = VGG16(weights='imagenet',include_top=False,input_shape =  t_x.shape[1:])

for layer in tmodel_base.layers[:-3]:
    layer.trainable=False

tmodel = Sequential()
tmodel.add(tmodel_base)
tmodel.add(Flatten())
tmodel.add(Dense(256,activation='relu',name='start_layer'))
tmodel.add(BatchNormalization())
tmodel.add(Dropout(0.10))
tmodel.add(Dense(1,activation='sigmoid',name='output_layer'))

tmodel.compile(optimizer=Adam(lr=1e-2),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])
tmodel.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weights_dir="{}_weights.best.hdf5".format('boat_detector')

checkpoint = ModelCheckpoint(weights_dir, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=15, verbose=1, mode='auto', epsilon=0.0001, cooldown=3, min_lr=0.0001)

#early = EarlyStopping(monitor="val_loss", 
                     # mode="min", 
                      #patience=10)

callbacks = [reduce,checkpoint]

In [ ]:
train_gen.batch_size = BATCH_SIZE
history = tmodel.fit_generator(train_gen, 
                         steps_per_epoch=train_gen.n//BATCH_SIZE,
                      validation_data=(valid_x, valid_y), 
                      epochs=30, 
                      callbacks=callbacks,
                      workers=4)

In [ ]:

tmodel.load_weights(weights_dir)
tmodel.save('tmodel_ship.h5')

In [ ]:
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

## Predict the test data
We use the sample_submission file as the basis for loading and running the images.

In [ ]:
test_paths = os.listdir(test_image_dir)
print(len(test_paths), 'test images found')
submission_df = pd.read_csv('../input/sample_submission_v2.csv')
submission_df['path'] = submission_df['ImageId'].map(lambda x: os.path.join(test_image_dir, x))

In [ ]:
test_gen = flow_from_dataframe(valid_idg, 
                               submission_df, 
                             path_col = 'path',
                            y_col = 'ImageId', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE, 
                              shuffle = False)

In [ ]:
test_gen = flow_from_dataframe(valid_idg, 
                               submission_df, 
                             path_col = 'path',
                            y_col = 'ImageId', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE, 
                              shuffle = False)

In [ ]:
from tqdm import tqdm_notebook
all_scores = dict()
for _, (t_x, t_names) in zip(tqdm_notebook(range(test_gen.n//BATCH_SIZE+1)),
                            test_gen):
    t_y = ship_model.predict(t_x)[:, 0]
    for c_id, c_score in zip(t_names, t_y):
        all_scores[c_id] = c_score

### Show the Scores

In [ ]:
submission_df['score'] = submission_df['ImageId'].map(lambda x: all_scores.get(x, 0))
submission_df['score'].hist()

### Make the RLE data if there is a ship
Here we make the RLE data for a positive image (assume every pixel is ship)

In [ ]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
# empty image
zp_dim = 10
out_img = np.ones((768-2*zp_dim, 768-2*zp_dim), dtype=bool)
out_img = np.pad(out_img, ((zp_dim, zp_dim),), mode='constant', constant_values=0)
plt.matshow(out_img)
print(out_img.shape)
pos_ship_str = rle_encode(out_img)
print(pos_ship_str[:50])

In [ ]:
# add the whole image if it is above the threshold
submission_df['EncodedPixels'] = submission_df['score'].map(lambda x: pos_ship_str if x>0.5 else None)

In [ ]:
out_df = submission_df[['ImageId', 'score']]
out_df.to_csv('submission.csv', index=False)
out_df.head(20)